# 트리 (Tree)

## 트리의 부모 찾기

- 문제 출처: [백준 11725번](https://www.acmicpc.net/problem/11725)

`-` 각 노드에 대해 모든 연결 노드를 그래프에 기록한다

`-` 루트 노드부터 BFS를 사용하자

`-` 탐색 과정에서 다음 노드의 부모는 현재 노드가 된다

In [1]:
from collections import defaultdict, deque


def solution():
    N = int(input())
    ROOT = 1
    graph = defaultdict(list)
    visited = {ROOT: True}
    node2parent = {}
    for _ in range(N - 1):
        a, b = map(int, input().split())
        graph[a].append(b)
        graph[b].append(a)
    queue = deque([ROOT])
    while queue:
        node = queue.popleft()
        for next_node in graph[node]:
            if next_node not in visited:
                visited[next_node] = True
                node2parent[next_node] = node
                queue.append(next_node)
    for node in range(2, N + 1):
        print(node2parent[node])


solution()

# input
# 12
# 1 2
# 1 3
# 2 4
# 3 5
# 3 6
# 4 7
# 4 8
# 5 9
# 5 10
# 6 11
# 6 12

 12
 1 2
 1 3
 2 4
 3 5
 3 6
 4 7
 4 8
 5 9
 5 10
 6 11
 6 12


1
1
2
3
3
4
4
5
5
6
6


## 트리 순회

- 문제 출처: [백준 1991번](https://www.acmicpc.net/problem/1991)

`-` 전형적인 트리 순회 문제

`-` 클래스와 재귀 함수를 이용해 깔끔하게 구현할 수 있다

`-` 내 코드보다 좋은 코드가 많이 있음 (배우자)

In [2]:
from collections import defaultdict


def preorder_traversal(tree, start, visited={}):
    visited[start] = True
    for next_node in tree[start]:
        if next_node != ABSENCE and next_node not in visited:
            preorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def inorder_traversal(tree, start, visited={}):
    left = tree[start][0]
    right = tree[start][1]
    if (left == ABSENCE or left in visited):
        visited[start] = True
    for next_node in [left, start, right]:
        if next_node != ABSENCE and next_node not in visited:
            inorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def postorder_traversal(tree, start, visited={}):
    left = tree[start][0]
    right = tree[start][1]
    if (left == ABSENCE or left in visited) and (right == ABSENCE or right in visited):
        visited[start] = True
    for next_node in [left, right, start]:
        if next_node != ABSENCE and next_node not in visited:
            postorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def solution():
    global ABSENCE
    N = int(input())
    tree = defaultdict(list)
    ROOT = "A"
    ABSENCE = "."
    for _ in range(N):
        parent, left, right = input().split()
        tree[parent].append(left)
        tree[parent].append(right)
    preorder_result = preorder_traversal(tree, ROOT)
    inorder_result = inorder_traversal(tree, ROOT)
    postorder_result = postorder_traversal(tree, ROOT)
    for result in [preorder_result, inorder_result, postorder_result]:
        print("".join(result))


solution()

# input
# 7
# A B C
# B D .
# C E F
# E . .
# F . G
# D . .
# G . .

 7
 A B C
 B D .
 C E F
 E . .
 F . G
 D . .
 G . .


ABDCEFG
DBAECFG
DBEGFCA


## 트리의 지름

- 문제 출처: [백준 19647번](https://www.acmicpc.net/problem/1967)

`-` [$\star$] `주의 사항: 이진 트리가 아닐 수도 있다` [$\star$]

`-` 트리의 지름이 되는 양 끝 노드를 생각해보면 둘은 자식 노드를 가질 수 없다

`-` $s$와 $t$가 트리의 지름이 된다고 해보자

`-` 만약 $s$에게 자식 노드 $u$가 있다면 $u\to s \to t$가 기존 트리의 지름보다 더 길다

`-` 이는 $s$와 $t$가 트리의 지름이 된다는 가정과 모순된다

`-` 따라서 자식이 없는 리프 노드만이 트리의 지름을 형성할 수 있다

`-` 리프 노드의 부모 노드를 고려하자

`-` 해당 부모 노드는 여러개의 리프 노드를 가진다

`-` 여러 리프 노드 중 부모와 연결된 간선의 가중치가 더 큰 리프 노드만 트리의 지름을 형성할 수 있다

`-` 부모 노드 $p$와의 가중치가 $w_1$인 $s_1$ 리프 노드가 다른 리프 노드 $u$와 트리의 지름을 형성한다고 해보자

`-` 해당 부모 노드의 또 다른 리프 노드 $s_2$는 $w_1$보다 큰 $w_2$의 가중치를 가진다고 하자

`-` 트리의 지름 $s_1 \to p \to u$보다 $s_2\to p \to u$가 더 큰 지름을 형성한다

`-` 따라서 여러 리프 노드 중 부모 노드와 연결된 간선의 가중치가 더 큰 리프 노드만 탐색하면 된다

`-` 단, 예외가 있는데 하나의 부모에 있는 두 리프 노드가 트리의 지름을 형성하면 위의 이론을 적용하면 안된다

`-` 이를 위해 부모와 연결된 리프 노드 중 간선의 가중치가 큰 순서대로 $2$개를 골라 더한 값을 전역 트리의 지름으로 갱신 시도해야 한다

`-` 아무튼 트리의 지름은 부모와 연결된 리프 노드를 잇는 선의 길이이다

`-` 그런데 이를 재귀적으로 반복할 수 있다

`-` 부모 노드 $p$와 리프 노드 $s_1, s_2,s_3$를 고려하자 (간선의 가중치가 큰 순서는 $s_1, s_2,s_3$라 하자) 

`-` 리프 노드 $s_1$이 선택됐다면 경로는 $p \to s_1$으로 고정되며 부모 노드 $p$는 사실상 리프 노드가 된다

`-` 트리의 지름은 $s_1 \to p \to s_2$와 $s_1\to p$를 지나는 다른 경로 중 더 큰 값으로 결정된다

`-` 초기에 모든 리프 노드에 대해 위의 작업을 수행하면 새로운 리프 노드가 생기고 이에 대해 반복하면 된다

- 알고리즘 요약

`1.` 루트 노드를 입력으로 넣는다

`2.` 입력된 노드의 자식 노드에 대해 함수를 재귀적으로 사용함

`3.` 함수 내에서 선택된 자식 노드와의 간선 가중치와 가장 큰 두 자식 노드를 잇는 트리의 지름을 기록함

`4.` 가장 큰 두 자식 노드를 잇는 트리의 지름을 트리의 지름 최댓값으로 갱신 시도

`5.` 선택된 자식 노드와의 간선 가중치를 기록하는 배열에 자식 노드 중 선택된 것을 추가로 더해준다

In [8]:
from collections import defaultdict


def dfs(start, tree, weight, child_weight):
    global MAX_DIAMETER
    edge_weights = []
    max_subtree_weight = 0
    for next_node in tree[start]:
        dfs(next_node, tree, weight, child_weight)
        w = weight[(start, next_node)] + child_weight[next_node]
        edge_weights.append(w)
        if w > max_subtree_weight:
            max_subtree_weight = w
    if len(edge_weights) > 2:
        edge_weights.sort(reverse=True)
        best_diameter = sum(edge_weights[:2])
    else:
        best_diameter = sum(edge_weights)
    MAX_DIAMETER = max(MAX_DIAMETER, best_diameter)
    child_weight[start] += max_subtree_weight


def solution():
    global MAX_DIAMETER
    n = int(input())
    tree = defaultdict(list)
    ROOT_NODE = 1
    MAX_DIAMETER = 0
    weight = {}
    child_weight = [0 for _ in range(n + 1)]
    for _ in range(n - 1):
        p, c, w = map(int, input().split())
        tree[p].append(c)
        weight[(p, c)] = w
    dfs(ROOT_NODE, tree, weight, child_weight)
    print(MAX_DIAMETER)


solution()

# input
# 12
# 1 2 3
# 1 3 2
# 2 4 5
# 3 5 11
# 3 6 9
# 4 7 1
# 4 8 7
# 5 9 15
# 5 10 4
# 6 11 6
# 6 12 10

 12
 1 2 3
 1 3 2
 2 4 5
 3 5 11
 3 6 9
 4 7 1
 4 8 7
 5 9 15
 5 10 4
 6 11 6
 6 12 10


45


`-` 참고로 트리의 지름을 구하는 또 다른 방법이 존재한다

`-` 임의의 한 노드에서 DFS를 사용해 가장 멀리 있는 노드를 찾고 해당 노드에서 DFS를 사용하면 트리의 지름을 구할 수 있다

## 트리의 지름

- 문제 출차: [백준 1167번](https://www.acmicpc.net/problem/1167)

`-` [트리의 지름](https://www.acmicpc.net/problem/1967) 문제의 심화 버전이다

`-` 나는 위의 문제를 이미 $O(N)$ 시간에 해결했기 때문에 비슷한 방법으로 이 문제도 풀 수 있다

`-` 차이점은 루트 노드가 주어지지 않고 부모 노드와 자식 노드 순서로 주어지지 않는 점이다

`-` 서로 연결된 노드는 서로의 인접 노드에 추가해야 한다

`-` 그리고 중복 방문을 허용하지 않는 코드를 DFS에 추가해야 한다

`-` 기존의 트리는 중복 방문을 할 일이 없었지만 지금은 그렇지 않다

`-` 기존 방식 대신 트리의 지름을 구하는 또 다른 방식을 사용해서 풀어볼까 한다

`-` 임의의 한 노드에서 DFS를 사용해 가장 멀리 있는 노드를 찾고 해당 노드에서 DFS를 사용하면 트리의 지름을 구할 수 있다

`-` 시간 복잡도는 $O(N)$으로 같다

`-` 위 방식이 성립하는 것을 증명해보자

- 알고리즘 증명

`-` $r$에서 DFS를 사용해 도착한 노드가 $u$라고 해보자

`-` 그리고 트리의 지름 양 끝 노드가 $x$와 $y$라고 해보자

`-` 트리의 노드는 서로 연결되어 있으며 두 노드를 잇는 경로는 하나만 존재한다

`-` 그러므로 만약 $r$이 $x$ 또는 $y$라면 자명하게 $u$는 $y$ 또는 $x$이다

`-` 이제 $r$이 $x$와 $y$ 둘 다 아니라고 해보자

`-` $u$가 $x$ 또는 $y$이면 당연히 성립하니 귀류법을 사용하고자 $u$가 전혀 다른 노드라고 해보자

`-` 그럼 $r \to u$ 경로가 있고 $x \to y$ 경로가 있을텐데 트리의 노드는 서로 연결되어 있다 했으므로 둘을 잇는 경로가 존재한다

`-` 배중률에 따라 두 경로를 모두 지나는 노드 $t$가 존재하거나 존재하지 않는다

`1.` 첫 번재로 $t$가 존재한다고 해보자

`-` 그러면 $r\to  t\to u$이고 $x\to t\to y$이다

`-` $r$에서 $u$가 가장 멀리 떨어져 있으므로 $t\to u$가 $t\to y$보다 길다

`-` 그렇지 않다면 $r$에서 가장 멀리 떨어진 노드는 $u$가 아니라 $y$가 되어야 한다

`-` 그런데 $t\to u$가 더 길기 때문에 $x\to t \to y$보다 $x\to t \to u$가 더 길다

`-` 즉, $u$는 $y$가 되어야 한다, 그렇지 않으면 모순이다

`2.` 두 번재로 $t$가 존재하지 않는다고 해보자

`-` 그러면 $r \to a \to u$이고 $x \to b \to y$이며 $a$와 $b$가 연결되어 있다고 해보자 (계속 언급했듯이 트리이므로 연결되어야만 한다)

`-` $u$가 $r$에서 가장 멀리 떨어졌으므로 $r \to a\to u$가 $r\to a\to b\to y$보다 길다

`-` 그런데 이게 사실이라면 트리의 지름은 $x\to b\to y$가 아니라 $x\to b \to a\to u$여야 하는데 이는 모순이다

`-` 따라서 이런한 경우는 존재할 수 없다

`-` 결과적으로 임의의 한 노드에서 가장 멀리 떨어진 노드는 트리의 지름의 양 끝 노드 중 하나이다

In [31]:
from collections import defaultdict


def dfs(node, graph, diameter=0, visited={}):
    global LAST_NODE, MAX_DIAMETER
    visited[node] = True
    for neighbor, weight in graph[node]:
        if neighbor not in visited:
            dfs(neighbor, graph, diameter + weight, visited)
    if diameter > MAX_DIAMETER:
        LAST_NODE = node  # 처음에 dfs에 입력한 노드와 가장 멀리 떨어진 노드
        MAX_DIAMETER = diameter


def solution():
    global LAST_NODE, MAX_DIAMETER
    V = int(input())
    graph = defaultdict(list)
    LAST_NODE = 1
    MAX_DIAMETER = 0
    start_node = 1
    for _ in range(V):
        u, *data, _ = map(int, input().split())
        for i in range(len(data) // 2):
            v, w = data[i * 2], data[i * 2 + 1]
            graph[u].append((v, w))
    dfs(start_node, graph, diameter=0, visited={})  # 트리의 지름을 잇는 두 점 중 하나를 탐색
    dfs(LAST_NODE, graph, diameter=0, visited={})  # 트리의 지름의 한 끝 점에서 나머지 끝 점을 탐색
    print(MAX_DIAMETER)


solution()

# input
# 5
# 1 3 2 -1
# 2 4 4 -1
# 3 1 2 4 3 -1
# 4 2 4 3 3 5 6 -1
# 5 4 6 -1

 5
 1 3 2 -1
 2 4 4 -1
 3 1 2 4 3 -1
 4 2 4 3 3 5 6 -1
 5 4 6 -1


11


`-` 주의 사항: dfs 함수의 visited 기본 값은 빈 딕셔너리로 되어 있다

`-` 인자의 기본 값으로 빈 리스트를 사용했을 때와 마찬가지의 부작용을 가지고 있으니 주의해야 한다

`-` visited 인자를 비어두고 함수를 다시 실행하면 전의 함수에서 갱신된 visited가 그대로 있다